In [1]:
import os
import json
import tensorflow as tf
from numpy import array
import random
random.seed( 3 )

from six.moves.urllib.request import urlretrieve

SOURCE_URL = 'http://pwz.mit.edu/data/train1018/'
WORK_DIRECTORY = "/train1018"

def maybe_download(filename):
    """A helper to download the data files if not present."""
    if not os.path.exists(WORK_DIRECTORY):
        os.mkdir(WORK_DIRECTORY)
    filepath = os.path.join(WORK_DIRECTORY, filename)
    if not os.path.exists(filepath):
        filepath, _ = urlretrieve(SOURCE_URL + filename, filepath)
        statinfo = os.stat(filepath)
        print('Succesfully downloaded', filename, statinfo.st_size, 'bytes.')
    else:
        print('Already downloaded', filename)
    return filepath  

In [2]:
data0 = maybe_download('0.txt')
data1 = maybe_download('1.txt')
data2 = maybe_download('2.txt')
data3 = maybe_download('3.txt')
data4 = maybe_download('4.txt')
data5 = maybe_download('5.txt')
data6 = maybe_download('6.txt')
data7 = maybe_download('7.txt')
data8 = maybe_download('8.txt')
data9 = maybe_download('9.txt')
data10 = maybe_download('10.txt')
data11 = maybe_download('11.txt')
data12 = maybe_download('12.txt')
data13 = maybe_download('13.txt')
data14 = maybe_download('14.txt')

test = maybe_download('test2.txt')




('Already downloaded', '0.txt')
('Already downloaded', '1.txt')
('Already downloaded', '2.txt')
('Already downloaded', '3.txt')
('Already downloaded', '4.txt')
('Already downloaded', '5.txt')
('Already downloaded', '6.txt')
('Already downloaded', '7.txt')
('Already downloaded', '8.txt')
('Already downloaded', '9.txt')
('Already downloaded', '10.txt')
('Already downloaded', '11.txt')
('Already downloaded', '12.txt')
('Already downloaded', '13.txt')
('Already downloaded', '14.txt')
('Already downloaded', 'test2.txt')


In [3]:
with open(data0, 'r') as f0,open(data1, 'r') as f1,open(data2, 'r') as f2,open(data3, 'r') as f3,open(data4, 'r') as f4,open(data5, 'r') as f5,open(data6, 'r') as f6,open(data7, 'r') as f7,open(data8, 'r') as f8,open(data9, 'r') as f9,open(data10, 'r') as f10,open(data11, 'r') as f11,open(data12, 'r') as f12,open(data13, 'r') as f13,open(data14, 'r') as f14,open(test, 'r') as ft:
    dd = []
    
    dd.append(json.load(f0))
    dd.append(json.load(f1))
    dd.append(json.load(f2))
    dd.append(json.load(f3))
    dd.append(json.load(f4))
    dd.append(json.load(f5))
    dd.append(json.load(f6))
    dd.append(json.load(f7))
    dd.append(json.load(f8))
    dd.append(json.load(f9))
    dd.append(json.load(f10))
    dd.append(json.load(f11))
    dd.append(json.load(f12))
    dd.append(json.load(f13))
    dd.append(json.load(f14))
    
    test_d = json.load(ft)
    
    print len(test_d),len(test_d[0]),len(test_d[0][0])


182 60 30


In [4]:
    train_data = []
    train_label = []
    test_data = []
    
    def combineSet(datamat,labelmat,data0,label):
        for img in data0:
            temp = []
            for row in img:
                for i in row:
                    temp.append(i)
            datamat.append(temp)
            labelmat.append(label)
    
    def labelcreator(index,total):
        out = [0 for i in range(total)]
        out[index]=1
        return out
            
    for i in range(15):
        combineSet(train_data,train_label,dd[i],labelcreator(i,15))

    def normalize(lst):
        return [i/5000 for i in lst]

    for img in test_d:
        temp = []
        for row in img:
            for i in row:
                temp.append(i)
        test_data.append(temp)


    #print normalize(train_data[0])
    train_data = [normalize(i) for i in train_data]
    test_data = [normalize(i) for i in test_data]

    
    #print array(train_data)
    print array(test_data)
    print len(test_data[0])
    #print len(train_data)

[[ 0.32        0.32188691  0.32766052 ...,  2.          2.          2.        ]
 [ 0.32        0.32188691  0.32766052 ...,  2.          2.          2.        ]
 [ 0.32        0.32188691  0.32766052 ...,  2.          2.          2.        ]
 ..., 
 [ 0.32        0.32188691  0.32766052 ...,  2.          2.          2.        ]
 [ 0.32        0.32188691  0.32766052 ...,  2.          2.          2.        ]
 [ 0.32        0.32188691  0.32766052 ...,  2.          2.          2.        ]]
1800


In [5]:
    
    setsize = 20
    setAll = 440
    
    def subset(lst,index,num):
        if index+num<len(lst):
            return lst[index:index+num]
        else:
            lst0 = lst[index:]
            lst1 = lst[:(num-(len(lst)-index))]
            lst0.extend(lst1)
            return lst0

    print len(subset(train_data,429,setsize))
    



20


In [6]:
    x = tf.placeholder(tf.float32,[None,1800])
    W = tf.Variable(tf.zeros([1800, 15])) # now have 3 labels, each pixel to each label have a weight
    b = tf.Variable(tf.zeros([15])) #each label have one bias
    y = tf.nn.softmax(tf.matmul(x, W) + b)
    y_ = tf.placeholder(tf.float32, [None, 15]) # the placeholder for the right labels

    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(cross_entropy)

    init = tf.initialize_all_variables()
    sess = tf.InteractiveSession()

    sess.run(init)                

In [7]:

for i in range(20000):
    myindex =  int(random.random()*setAll)
    if i%100 == 0:
        print i
    batch_xs, batch_ys = array(subset(train_data,myindex,setsize)), array(subset(train_label,myindex,setsize))
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

#print array(train_data[0]),array(test_data[0])
print sess.run(W)
print sess.run(b)
#correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [8]:

sys_prediction = y

#print len(test_data),len(test_data[0])
print sess.run(sys_prediction, feed_dict={x: array(test_data)})

import numpy as np

#print sys_prediction.eval(sess,feed_dict={x: array(test_data)})
out = "["

for (x,y), value in np.ndenumerate(sess.run(sys_prediction, feed_dict={x: array(test_data)})):
    if y == 0:
        #print x
        if x != 0:
            out +="]"
        out += "["

    out += str(value)+","

out += "]]"

with open("test2_result.txt", "w") as text_file:
    text_file.write(out)


[[  3.48626524e-02   4.29959316e-03   2.05136384e-05 ...,   1.62510405e-05
    1.62510405e-05   1.62510405e-05]
 [  2.98271663e-02   3.61353345e-03   1.63133991e-05 ...,   1.64549474e-05
    1.64549474e-05   1.64549474e-05]
 [  3.18756476e-02   3.79820517e-03   1.69458490e-05 ...,   1.78287064e-05
    1.78287064e-05   1.78287064e-05]
 ..., 
 [  5.88528156e-01   2.87454039e-01   2.41722260e-03 ...,   6.45742875e-06
    6.45742875e-06   6.45742875e-06]
 [  6.00927949e-01   3.24129164e-01   2.93317228e-03 ...,   4.64279356e-06
    4.64279356e-06   4.64279356e-06]
 [  5.97398937e-01   3.56562495e-01   3.39521025e-03 ...,   3.32107493e-06
    3.32107516e-06   3.32107516e-06]]


In [9]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Save the variables to disk.
save_path = saver.save(sess, "/model_test.ckpt")
print("Model saved in file: %s" % save_path)


Model saved in file: /model_test.ckpt


In [12]:
saver = tf.train.Saver()

saver.restore(sess, "/model_test.ckpt")
print("Model restored.")


NotFoundError: /sys/devices/system/container/ACPI0004:00/VMBUS:00/firmware_node/vmbus_122

In [8]:
sys_prediction = y


#print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


def byteify(input):
    if isinstance(input, dict):
        return {byteify(key): byteify(value)
                for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [byteify(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input


out_0 = "["
out_1 = "["
out_2 = "["
out_3 = "["
out_4 = "["
out_5 = "["
out_6 = "["
out_7 = "["
out_8 = "["
out_9 = "["
out_10 = "["

for (x,y), value in np.ndenumerate(W.eval(sess)):
    
    if y == 0:
        out_0 += str(value)+","
        print x
    if y == 1:
        out_1 += str(value)+","
    if y == 2:
        out_2 += str(value)+","
    if y == 3:
        out_3 += str(value)+","
    if y == 4:
        out_4 += str(value)+","
    if y == 5:
        out_5 += str(value)+","
    if y == 6:
        out_6 += str(value)+","
    if y == 7:
        out_7 += str(value)+","
    if y == 8:
        out_8 += str(value)+","
    if y == 9:
        out_9 += str(value)+","
    if y == 10:
        out_10 += str(value)+","

out_0 += "]"
out_1 += "]"
out_2 += "]"
out_3 += "]"
out_4 += "]"
out_5 += "]"
out_6 += "]"
out_7 += "]"
out_8 += "]"
out_9 += "]"
out_10 += "]"


with open("train1018/weight_0.txt", "w") as text_file:
    text_file.write(out_0)
with open("train1018/weight_1.txt", "w") as text_file:
    text_file.write(out_1)
with open("train1018/weight_2.txt", "w") as text_file:
    text_file.write(out_2)
with open("train1018/weight_3.txt", "w") as text_file:
    text_file.write(out_3)
with open("train1018/weight_4.txt", "w") as text_file:
    text_file.write(out_4)
with open("train1018/weight_5.txt", "w") as text_file:
    text_file.write(out_5)
with open("train1018/weight_6.txt", "w") as text_file:
    text_file.write(out_6)
with open("train1018/weight_7.txt", "w") as text_file:
    text_file.write(out_7)
with open("train1018/weight_8.txt", "w") as text_file:
    text_file.write(out_8)
with open("train1018/weight_9.txt", "w") as text_file:
    text_file.write(out_9)
with open("train1018/weight_10.txt", "w") as text_file:
    text_file.write(out_10)



[[ 0.32        0.32188691  0.32766052 ...,  0.3481393   0.34200485  0.34      ]
 [ 0.32        0.32188691  0.32766052 ...,  0.3481393   0.34200485  0.34      ]
 [ 0.32        0.32188691  0.32766052 ...,  0.3481393   0.34200485  0.34      ]
 ..., 
 [ 0.32        0.32188691  0.32766052 ...,  0.18430904  0.18106139  0.18      ]
 [ 0.32        0.32188691  0.32766052 ...,  0.18430904  0.18106139  0.18      ]
 [ 0.32        0.32188691  0.32766052 ...,  0.18430904  0.18106139  0.18      ]]
[[  5.28126862e-03   4.82030120e-03   8.18005484e-03 ...,   1.77889749e-01
    3.13160381e-05   7.85543680e-01]
 [  5.83190285e-03   3.85313109e-03   9.66982357e-03 ...,   2.33140305e-01
    1.36631998e-05   7.20053315e-01]
 [  6.85363635e-03   3.39043140e-03   1.08535299e-02 ...,   2.62514204e-01
    9.55679570e-06   6.78331792e-01]
 ..., 
 [  8.04059207e-03   2.82377866e-03   5.45617677e-02 ...,   2.51793921e-01
    2.22967742e-06   5.94801068e-01]
 [  5.92201250e-03   2.09113001e-03   3.03253736e-02 ...,